# BHIT (Numpy)

This notebook uses pure python and **Numpy** package to run BHIT.

## Import Necessary Packages

1. `time` package for measuring running time of the whole program;
2. `scipy.speical` package for calculating logrithmic gamma function;
3. `numpy` package for matrix manipulation.

In [1]:
import time
import scipy.special
import numpy as np

## Define Some Helper Functions

### Function init()
This function intializes parameters that will be used in the program.
#### Parameters
- `dataset`: The whole dataset read from the input file.
- `outFileName`: Name of output file.
- `iterNum`: Number of iteration.
- `burninNum`: Burn-in number of iterations.
- `ObsNum`: Number of observations, e.g., number of population.
- `SNPNum`: Number of single nucleotide polymorphisms (SNPs).
- `PhenoNum`: Number of phenotype types.
- `MAF`: Minor Allele Frequency, should be less than 1.

** There are some differences between Jupyter notebook version and command line version! **

*We define parameter values inside the function instead of reading them from command line, because Jupyter cannot run with self-defined parameters.*

In [2]:
def init():
    
    dataset = np.loadtxt('input.txt')
    outFileName = 'output.npy'
    iterNum = 30000
    burninNum = 29000
    ObsNum = 200
    SNPNum = 100
    PhenoNum = 1
    MAF = 0.5
    
    return dataset, outFileName, iterNum, burninNum, ObsNum, SNPNum, PhenoNum, MAF


### Funcation unique(arr, return_counts=True)
This function finds the unique elements of an array. It returns the sorted unique elements of an array, and the number of times each unique value comes up in the input array if `return_counts` is True.
#### Parameters
- `arr`: Input array.
- `return_counts`: If `True`, also return the number of times each unique item appears in `arr`.

#### Returns
- `unique`: The sorted unique values.
- `unique_counts`: The number of times each of the unique values comes up in the original array.

This function is written based on Numpy v1.14 unique() function. Considering that **low version may not support extra parameters**, like `return_counts` we need in this problem, we rewrote the unique() function for our specific usage.

*However, if you install Numpy v1.14 on your machine, you may delete this function to see if the program works properly. *



In [3]:
def unique(arr, return_counts=True):
    
    arr = np.asanyarray(arr)    
    orig_shape, orig_dtype = arr.shape, arr.dtype
    # Must reshape to a contiguous 2D array for this to work...
    arr = arr.reshape(orig_shape[0], -1)
    arr = np.ascontiguousarray(arr)

    if arr.dtype.char in (np.typecodes['AllInteger'] +
                         np.typecodes['Datetime'] + 'S'):
        # Optimization: Creating a view of your data with a np.void data type of
        # size the number of bytes in a full row. Handles any type where items
        # have a unique binary representation, i.e. 0 is only 0, not +0 and -0.
        dtype = np.dtype((np.void, arr.dtype.itemsize * arr.shape[1]))
    else:
        dtype = [('f{i}'.format(i=i), arr.dtype) for i in range(arr.shape[1])]

    try:
        consolidated = arr.view(dtype)
    except TypeError:
        # There's no good way to do this for object arrays, etc...
        msg = 'The axis argument to unique is not supported for dtype {dt}'
        raise TypeError(msg.format(dt=arr.dtype))

    def reshape_uniq(uniq):
        uniq = uniq.view(orig_dtype)
        uniq = uniq.reshape(-1, *orig_shape[1:])
        uniq = np.swapaxes(uniq, 0, 0)
        return uniq

    tmp = np.asanyarray(consolidated).flatten()

    if tmp.size == 0:
        if not return_counts:
            output = tmp
        else:
            output = (tmp,)
            output += (np.empty(0, np.intp),)
    else:
        tmp.sort()
        aux = tmp
        flag = np.concatenate(([True], aux[1:] != aux[:-1]))
        
        if not return_counts:
            output = aux[flag]
        else:
            output = (aux[flag],)
            idx = np.concatenate(np.nonzero(flag) + ([tmp.size],))
            output += (np.diff(idx),)
    
    if not return_counts:
        return reshape_uniq(output)
    else:
        uniq = reshape_uniq(output[0])
        return (uniq,) + output[1:]


### Function logLikeCont(contArr)
This function calculates logarithmic likelihood of only continuous variates given the data array.
#### Math formulas
We assume $Gaussian$ distribution on the continuous data, and the probability is:

$$P(Y\mid X) = (\frac{1}{2\pi})^n\sqrt{\frac{\kappa_0}{\kappa_n}}\frac{\Gamma(v_n/2)}{\Gamma(v_0/2)}((\frac{v_0\sigma_0^2}{2})^{v_0/2}/(\frac{v_n\sigma_n^2}{2})^{v_n/2})$$

where

- $\kappa_n = \kappa_0 + n$
- $v_n = v_0 + n$
- $\sigma_n^2 = \frac{1}{v_n}(v_0\sigma_0^2+(n-1)s^2+\frac{\kappa_0 n}{\kappa_n}(\bar y - \mu_0)^2)$
- $s^2 = \frac{1}{n-1}\sum_{i=1}^n(y_i-\bar y)^2$
- $\kappa_0, \mu_0, \sigma_0, v_0$ is pre-defined in the program ([click here](#Run-This-Program)), $n$ is the observation number of continuous data

#### Parameters
- `contArr`: Input array.

#### Returns
- `logProb`: Logarithmic likelihood of continuous variates.

In [4]:
def logLikeCont(contArr):
    
    Y = np.asanyarray(contArr)
    obsNum, varNum = Y.shape
    logProb = 0
    
    if varNum == 0:
        pass
    elif varNum == 1:
        sigma = 1
        mean = np.average(Y)
        nuVar = NU0 * sigma**2
        nuVar += (obsNum-1) * np.var(Y)
        nuVar += KAPPA0*obsNum/(KAPPA0+obsNum) * (mean-MU0)**2

        logProb = -1*np.log(2*np.pi)*obsNum/2 + np.log(KAPPA0/(KAPPA0 + obsNum)) / 2 + scipy.special.gammaln((NU0+obsNum)/2)
        logProb += (-1*scipy.special.gammaln(NU0/2) + np.log(NU0*sigma**2/2)*NU0/2 - np.log(nuVar/2) * (NU0+obsNum)/2)
    
    # The below code was not fully tested.
    else:
        means = np.average(Y, axis=0)
        lambda_arr = np.diag([1]*varNum)
        diff = np.array(means - MU0)[:,None]
        lambdaN = (lambda_arr + KAPPA0 * obsNum / (KAPPA0+obsNum)
                    * diff.dot(diff.transpose()))
        lambdaN += (obsNum-1)*np.cov(Y, rowvar=False, bias=False)

        logProb = (-np.log(np.pi) * obsNum * varNum / 2 + np.log(KAPPA0/
            (KAPPA0 + obsNum) * varNum / 2))
        logProb += np.log(np.linalg.det(lambda_arr)) * NU0/2
        logProb -= np.log(np.linalg.det(lambdaN)) * (NU0+obsNum)/2
        logProb += np.sum(scipy.special.gammaln((NU0+obsNum)/2 - 
            np.arange(varNum)/2) - scipy.special.gammaln(NU0/2 - 
            np.arange(varNum)/2))
        
    return logProb


### Function logLikeDisc(discArr)
This function calculates logarithmic likelihood of only discrete variates given the data array.
#### Math formulas
We assume $Dirichlet$ distribution on the discrete data, and the probability is:

$$P(p_1, \cdots, p_{C_h}\mid \alpha_1, \cdots, \alpha_{C_h}) = \frac{1}{B(\alpha)}\prod_{j=1}^{C_h}p_{j}^{\alpha_{j-1}} $$

where 

- $C_h$ is the possible combination values in the genetic variation group (incoming parameter `discArr`)
- $\alpha = (\alpha_1, \cdots, \alpha_{C_h})$, parameter vector in $Dirichlet$ distribution
- $B(\alpha) = \frac{\prod_{j=1}^{C_h}\Gamma(\alpha_j)}{\Gamma(\sum_{j=1}^{C_h}\alpha_j)}$


By integrating these, we can get following formula:
$$ P(X\mid I) = \prod_{j=1}^{C_h}\frac{\Gamma(n_j+\alpha_j)}{\Gamma(\alpha_j)}\frac{\Gamma(\sum_{j=1}^{C_h}\alpha_j)}{\Gamma(\sum_{j=1}^{C_h}(n_j+\alpha_j))}$$

where 

- $X$ is the genetic variation group, `discArr` in the program
- $I$ is current partition, `Ix` or `Iy` in the program
- $n_j$ denotes the number of j-th value in possible combination shown up in the genetic variation group $X$
- $\Gamma (x) = \int_0^{\infty}t^{x-1}e^{-t}dt$.

#### Parameters:
- `discArr`: Input array.

#### Returns
- `logProb`: Logarithmic likelihood of discrete variates.

In [5]:
def logLikeDisc(discArr):

    X = np.asanyarray(discArr)
    uniqueArr, N = unique(X)
    
    alpha = odds[uniqueArr-1]
    alpha = np.prod(alpha, axis=1)
    n_plus_alpha = N + alpha
    
    logProb = np.sum(scipy.special.gammaln(n_plus_alpha) - scipy.special.gammaln(alpha))
    logProb -= scipy.special.gammaln(np.sum(n_plus_alpha))
    return logProb


### Function logLikeDepe(discArr, contArr)
This function calculates logarithmic likelihood of partitions with both continuous and discrete variates.
#### Math formulas
If we detect interaction between both continuous and discrete data, we calculate probability as follows:
$$P = \prod_{m=1}^{M}P(Y_{\{m\}} \mid X_{\{I=h\}}) P({X_{\{I=h\}}}\mid I)$$

where

- $M$ is the total number of combination values of $X$ that are associated with $Y$
- The formula can be calculated by combining two formulas we defined above

#### Parameters
- `discArr`: Input discrete array.
- `contArr`: Input continous array.
#### Returns
- `logProb`: Logarithmic likelihood of both continuous and discrete variates.

In [6]:
def logLikeDepe(discArr, contArr):
    
    X = np.asanyarray(discArr)
    Y = np.asanyarray(contArr)
    variations = unique(X, return_counts=False)
    logProb = 0
    
    for v in variations:
        corres_row = np.prod((X==v), axis=1)
        corres_Y = Y[corres_row==1]
        logProb += logLikeCont(corres_Y)
    logProb += logLikeDisc(X)

    return logProb


### Function metroHast(iterNum, burninNum)

The function uses Metropolis–Hastings algorithm for sampling, and runs as follows:
1. Initialize index vector to be $[1, \cdots, n]$, where $n$ is the sum of SNP number and phenotype number;
2. Select one index from the vector and change it to another one randomly;
3. Calculate likelihood of original index partition vector, i.e., `old_prob` in the program;
4. Calculate likelihood of new index partition vector, i.e., `new_prob` in the program;
5. If the later likelihood is greater than the former, we update the index partition vector and other relevant staff (actually we might also update under some situations).

#### Parameters
- `iterNum`: Number of iteration of MCMC.
- `burninNum`: Number of burn-in of MCMC.

#### Returns
- `mhResult`: Final partition matrix for each covariate.
- `Ix`: Final index vector.

In [7]:
def metroHast(iterNum, burninNum):
    
    Ix = np.arange(TotalNum)
    Dx = Ix[:SNPNum]
    Cx = Ix[SNPNum:TotalNum]
    iN = np.zeros([TotalNum, TotalNum+1])
    
    # Uncomment if you want to trace probabilities.
    # trace = np.zeros(iterNum)
    # trace[0] += np.sum([logLikDisc(genotype[:, Dx==col]) for col in range(SNPNum)])
    # trace[0] += np.sum([logLikCont(phenotype[:, Cx==col]) for col in range(SNPNum,TotalNum)])

    # Main Metropolis-Hastings loop.
    for i in range(1, iterNum):
        # Select an index, then change it to another index randomly.
        while True:
            # Sort the number to ensure changing from small index to big one.
            x, y = np.sort(np.random.choice(Ix, 2, False))
            k = np.where(Ix == x)[0]
            
            if len(k) > 1:
                k = np.random.choice(k, 1)
          
            Iy = np.array(Ix)
            Iy[k] = y
          
            tmp1 = np.where(Ix == x)[0]
            tmp2 = np.where(Iy == y)[0]
            if (len(tmp1)!=1 or len(tmp2)!=1):
                break

        # Create the proposed indicator vector.
        Dy = Iy[:SNPNum]
        Cy = Iy[SNPNum:TotalNum]
        Cxx = phenotype[:,Cx == x]
        Cxy = phenotype[:,Cx == y]
        Cyx = phenotype[:,Cy == x]
        Cyy = phenotype[:,Cy == y]
        Dxx = genotype[:,Dx == x]
        Dxy = genotype[:,Dx == y]
        Dyx = genotype[:,Dy == x]
        Dyy = genotype[:,Dy == y]
        
        # Calculate log likelihoods.
        old_prob = 0
        new_prob = 0
        
        # Likelihood of current partition x.
        if Cxx.size != 0:
            if Dxx.size != 0:
                old_prob += logLikeDepe(Dxx, Cxx)
            else:
                old_prob += logLikeCont(Cxx)
        elif Dxx.size != 0:
            old_prob += logLikeDisc(Dxx)
        
        # Likelihood of current partition x.
        if Cxy.size != 0:
            if Dxy.size != 0:
                old_prob += logLikeDepe(Dxy, Cxy)
            else:
                old_prob += logLikeCont(Cxy)
        elif Dxy.size != 0:
            old_prob += logLikeDisc(Dxy)
        
        # Likelihood of proposed partition y.
        if Cyx.size != 0:
            if Dyx.size != 0:
                new_prob += logLikeDepe(Dyx, Cyx)
            else:
                new_prob += logLikeCont(Cyx)
        elif Dyx.size != 0:
            new_prob += logLikeDisc(Dyx)

        # Likelihood of proposed partition y.
        if Cyy.size != 0:
            if Dyy.size != 0:
                new_prob += logLikeDepe(Dyy, Cyy)
            else:
                new_prob += logLikeCont(Cyy)
        elif Dyy.size != 0:
            new_prob += logLikeDisc(Dyy)
        
        # Uncomment if you want to trace probabilities.
        # trace[i] = trace[i-1]
        
        # Check if proposal is accepted, if so, update everything.
        accept = np.log(np.random.rand()) <= min(0.0, new_prob-old_prob)
        if accept:
            Ix = np.array(Iy)
            Cx = np.array(Cy)
            Dx = np.array(Dy)
            # trace[i] += new_prob-old_prob
        
        if (i+1) % 5000 == 0:
            print("Progress: %.2f%%" % ((i+1)/iterNum*100))
            
        # When MCMC gets convergence, we start to count indices.
        if i >= burninNum:
            for h in range(TotalNum):
                tmp = np.where(Ix==h)[0]
                if (len(tmp) == 1):
                    iN[tmp, 0] += 1
                elif (len(tmp) > 1):
                    iN[tmp, h+1] += 1

    # Normalize rows of result to show proportions.
    iN = iN/np.sum(iN, axis=1)[:,None]
    cst = np.sum(iN, axis=0)[1:]

    # Remove columns that have no values in them.
    tmp = np.where(cst != 0)[0]
    iNt = iN[:, 1:]
    if len(tmp) == 0:
        iNtr = iNt[:,1]
    else:
        iNtr = iNt[:,tmp]
    
    # Have the first column correspond to all independent variates, 
    # and others are partitions with more than one variate.
    mhResult = np.zeros([TotalNum+1, len(tmp)+2])
    mhResult[0, 1:] = np.arange(len(tmp)+1)
    mhResult[1:SNPNum+1, 0] = np.arange(1, SNPNum+1)
    mhResult[SNPNum+1:, 0] = np.arange(1, PhenoNum+1)
    mhResult[1:,1] = iN[:TotalNum,0]
    mhResult[1:,2:] = iNtr
    
    # Uncomment if you want to save trace file.
    # np.savetxt(outFileName+"_trace", trace, fmt='%.1f')
    
    return mhResult, Ix


## Run This Program

In [8]:
start = time.time()

# Initialize parameters for later usage.
(dataset, outFileName, iterNum, burninNum, 
 ObsNum, SNPNum, PhenoNum, MAF) = init()
TotalNum = SNPNum + PhenoNum
genotype = dataset[:, :SNPNum].astype(int)
phenotype = dataset[:, SNPNum:TotalNum]
odds = np.array([(1-MAF)**2, 2*MAF*(1-MAF), MAF**2])

# Define hyper-parameters here.
KAPPA0 = 1
NU0 = PhenoNum + 1
MEANS = np.average(phenotype, axis=0)
MU0 = max(MEANS) + 2

# Check if input file format is valid.
if (genotype.shape[0] != phenotype.shape[0]):
    print("Discrete and continuous data must have same number of rows!\n")
    quit()
print("Initialization Completed!")

# Detection using Metropolis–Hastings algorithm.
mhResult, index = metroHast(iterNum, burninNum)
np.savetxt(outFileName, mhResult, fmt='%i')

# Output running time of the program.
end = time.time()
print("The whole program runs about %.2fs." % (end-start))


Initialization Completed!
Progress: 16.67%
Progress: 33.33%
Progress: 50.00%
Progress: 66.67%
Progress: 83.33%
Progress: 100.00%
The whole program runs about 42.86s.


## Results

The function `metroHast` actually returns two values: one is the final index vector, and the other is the independence array of each variate.

In [9]:
print(mhResult)

[[  0.   0.   1.   2.]
 [  1.   0.   0.   1.]
 [  2.   0.   0.   1.]
 [  3.   1.   0.   0.]
 [  4.   1.   0.   0.]
 [  5.   1.   0.   0.]
 [  6.   1.   0.   0.]
 [  7.   1.   0.   0.]
 [  8.   1.   0.   0.]
 [  9.   1.   0.   0.]
 [ 10.   1.   0.   0.]
 [ 11.   1.   0.   0.]
 [ 12.   1.   0.   0.]
 [ 13.   1.   0.   0.]
 [ 14.   1.   0.   0.]
 [ 15.   1.   0.   0.]
 [ 16.   1.   0.   0.]
 [ 17.   1.   0.   0.]
 [ 18.   1.   0.   0.]
 [ 19.   1.   0.   0.]
 [ 20.   1.   0.   0.]
 [ 21.   1.   0.   0.]
 [ 22.   1.   0.   0.]
 [ 23.   1.   0.   0.]
 [ 24.   1.   0.   0.]
 [ 25.   1.   0.   0.]
 [ 26.   1.   0.   0.]
 [ 27.   1.   0.   0.]
 [ 28.   1.   0.   0.]
 [ 29.   1.   0.   0.]
 [ 30.   1.   0.   0.]
 [ 31.   1.   0.   0.]
 [ 32.   1.   0.   0.]
 [ 33.   1.   0.   0.]
 [ 34.   1.   0.   0.]
 [ 35.   1.   0.   0.]
 [ 36.   1.   0.   0.]
 [ 37.   1.   0.   0.]
 [ 38.   1.   0.   0.]
 [ 39.   1.   0.   0.]
 [ 40.   1.   0.   0.]
 [ 41.   1.   0.   0.]
 [ 42.   1.   0.   0.]
 [ 43.   1.

The first row of `mhResult` is an indicator row, which tells us how many interactions detected in the dataset. In this example it is 2. The second column tells independent variates in the dataset. As we only care about the relation between genotype and phenotype, the above result is accepted.

In [10]:
print(index)

[100 100   2   3   4   5   6   7   8   9  39  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  90  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  97  91  92  93  94  95  96  97  98  99 100]


As we can see from final index vector, there are some indices changed during the process. For example, both the first and second value are changed to 100.